In [1]:
#Author: Nicolas Martin 
#This is a low-memory version of Flux for GPUs<16GB,
#use image fractalapps/genimages:v0815a to have everything ready (see readme.md)

#As Flux Dev is an open source version, its results are not so good as the dev and the pro versions.
#Nevertheless, you can generate several renders to improve your chances to have a great result.

import torch

from diffusers import  FluxPipeline

from datetime import datetime

import random


import time

from optimum.quanto import freeze, qfloat8, quantize

from diffusers import FlowMatchEulerDiscreteScheduler, AutoencoderKL
from diffusers.models.transformers.transformer_flux import FluxTransformer2DModel
from diffusers.pipelines.flux.pipeline_flux import FluxPipeline
from transformers import CLIPTextModel, CLIPTokenizer,T5EncoderModel, T5TokenizerFast


/usr/local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.
0it [00:00, ?it/s]
/usr/local/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  4.43it/s]
You set `add_prefix_space`. The tokenizer

In [ ]:
#Source: https://gist.github.com/VvanGemert/ab9c3ce63f12d429cf6075dbd764e57c

dtype = torch.bfloat16

# schnell is the distilled turbo model. For the CFG distilled model, use:
# bfl_repo = "black-forest-labs/FLUX.1-dev"
# revision = "refs/pr/3"
#

#bfl_repo = "black-forest-labs/FLUX.1-schnell"
bfl_repo = "/gen_model/"
revision = "refs/pr/1"

scheduler = FlowMatchEulerDiscreteScheduler.from_pretrained(bfl_repo, subfolder="scheduler", revision=revision)
text_encoder = CLIPTextModel.from_pretrained("openai/clip-vit-large-patch14", torch_dtype=dtype)
tokenizer = CLIPTokenizer.from_pretrained("openai/clip-vit-large-patch14", torch_dtype=dtype)
text_encoder_2 = T5EncoderModel.from_pretrained(bfl_repo, subfolder="text_encoder_2", torch_dtype=dtype, revision=revision)
tokenizer_2 = T5TokenizerFast.from_pretrained(bfl_repo, subfolder="tokenizer_2", torch_dtype=dtype, revision=revision)
vae = AutoencoderKL.from_pretrained(bfl_repo, subfolder="vae", torch_dtype=dtype, revision=revision)
transformer = FluxTransformer2DModel.from_pretrained(bfl_repo, subfolder="transformer", torch_dtype=dtype, revision=revision)

# Experimental: Try this to load in 4-bit for <16GB cards.
#
# from optimum.quanto import qint4
# quantize(transformer, weights=qint4, exclude=["proj_out", "x_embedder", "norm_out", "context_embedder"])
# freeze(transformer)
quantize(transformer, weights=qfloat8)
freeze(transformer)

quantize(text_encoder_2, weights=qfloat8)
freeze(text_encoder_2)

pipe = FluxPipeline(
    scheduler=scheduler,
    text_encoder=text_encoder,
    tokenizer=tokenizer,
    text_encoder_2=None,
    tokenizer_2=tokenizer_2,
    vae=vae,
    transformer=None,
)
pipe.text_encoder_2 = text_encoder_2
pipe.transformer = transformer
pipe.enable_model_cpu_offload()

In [1]:
#Picture's dimensions
height=768
width=1360

In [16]:
#Inference part A

#Here, we create several random renders from the same prompt with a precise naming based on the seed and the date.


prompt = "hyper realistic Earth from satellite centered on India"

renders = 10



for i in range(0,renders):

    seed = random.randint(10000, 99999)

    print(seed)
    
    out = pipe(
        prompt=prompt,
        guidance_scale=0.,
        height=height,
        width=width,
        num_inference_steps=4,
        generator=torch.Generator("cuda").manual_seed(seed),
        max_sequence_length=256,
    ).images[0]
    
    # Get the current date and time
    now = datetime.now()
    
    # Format the date and time as Month_Day_HH_MM_SS
    formatted_time = now.strftime("%m_%d_%H_%M_%S")
    
    image_name = formatted_time + "_" + str(seed) + "_" + str(i) + "_A.png"
    out.save(image_name)
    
    time.sleep(1)

print("finished")

72805


100%|██████████| 4/4 [00:05<00:00,  1.47s/it]


90872


100%|██████████| 4/4 [00:05<00:00,  1.44s/it]


53201


...

48880


100%|██████████| 4/4 [00:06<00:00,  1.51s/it]


64593


100%|██████████| 4/4 [00:05<00:00,  1.49s/it]


finished


In [21]:
#Inference part B

prompt = 'a sport motorbike with "CHECK" printed in green, hyper realistic, exposition background'

renders = 6

for i in range(0,renders):

    seed = random.randint(10000, 99999) #15000 + i #

    print(seed)
    
    out = pipe(
        prompt=prompt,
        guidance_scale=0.,
        height=height,
        width=width,
        num_inference_steps=4,
        generator=torch.Generator("cpu").manual_seed(seed),
        max_sequence_length=256,
    ).images[0]
    
    # Get the current date and time
    now = datetime.now()
    
    # Format the date and time as Month_Day_HH_MM_SS
    formatted_time = now.strftime("%m_%d_%H_%M_%S")
    
    image_name = formatted_time + "_" + str(seed) + "_" + str(i) + "_B.png"
    out.save(image_name)
    
    time.sleep(2)

print("finished")

32725


100%|██████████| 4/4 [00:05<00:00,  1.45s/it]


31893


100%|██████████| 4/4 [00:05<00:00,  1.47s/it]


27685


...

16764


100%|██████████| 4/4 [00:05<00:00,  1.47s/it]


78097


100%|██████████| 4/4 [00:05<00:00,  1.46s/it]


finished


In [12]:
#Inference part C

prompt = 'A futuristic rocket takes off, high angle, extreme close up from top, fire effects, ground background, hyper realistic'

renders = 7


for i in range(0,renders):

    seed = random.randint(10000, 99999) #15000 + i #

    print(seed)
    
    out = pipe(
        prompt=prompt,
        guidance_scale=0.,
        height=height,
        width=width,
        num_inference_steps=4,
        generator=torch.Generator("cpu").manual_seed(seed),
        max_sequence_length=256,
    ).images[0]
    
    # Get the current date and time
    now = datetime.now()
    
    # Format the date and time as Month_Day_HH_MM_SS
    formatted_time = now.strftime("%m_%d_%H_%M_%S")
    
    image_name = formatted_time + "_" + str(seed) + "_" + str(i) + "_C.png"
    out.save(image_name)
    
    time.sleep(2)

print("finished")


63548


100%|██████████| 4/4 [00:05<00:00,  1.44s/it]


65924


100%|██████████| 4/4 [00:05<00:00,  1.45s/it]


96080


...

97073


100%|██████████| 4/4 [00:05<00:00,  1.48s/it]


63426


100%|██████████| 4/4 [00:05<00:00,  1.47s/it]


finished
